In [1]:
!python3 -m pip install --upgrade pip
!pip install python-dotenv tiktoken
!pip install openai==1.45.0
!pip install transformers accelerate sentencepiece bitsandbytes faiss-gpu
!pip install tabulate==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.0 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 1.1 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: openai
    Found existing installation: openai 1.30.0
    Uninstalling openai-1.30.0:
      Successfully uninstalled openai-1.30.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 1.0 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 1.2 MB/s eta

In [2]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from tabulate import tabulate
import tiktoken
import faiss
import openai
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
# OpenAIのAPIキーを設定
openai.api_key = os.getenv("OPENAI_API_KEY")

MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
# MODEL_NAME = "gpt-4-0125-preview"
# MODEL_NAME = "gpt-4-turbo-2024-04-09"
MODEL4o_NAME = "gpt-4o-2024-05-13"

# モデルとエンコーディングの設定
# embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"

# max_tokens = 8000
# max_tokens = 1536
# max_tokens = 128
max_tokens = 3072

# EMBEDDING_MODEL = "text-embedding-ada-002"
# EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_MODEL = "text-embedding-3-large"
TEMPERATURE = 0.7

# OpenAIクライアントの初期化
client = OpenAI()

In [5]:
# エンコーディングの取得
encoding = tiktoken.get_encoding(embedding_encoding)

In [6]:
# パスの定義
data_path = Path('../input')
novel_data_path = data_path / 'novels' / 'novels'
output_dir = data_path / 'processed_novels'  # 前処理済みのテキストファイルが格納されているディレクトリ
csv_output_dir = data_path / 'embeddings_csv'  # CSVファイルの出力先ディレクトリ

In [7]:
# 出力ディレクトリの作成
output_dir.mkdir(parents=True, exist_ok=True)
csv_output_dir.mkdir(parents=True, exist_ok=True)

In [8]:
# テキストをチャンクに分割する関数
def chunk_text(text, max_tokens, encoding):
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = encoding.decode(chunk_tokens)
        chunks.append(chunk_text)
    return chunks

In [9]:
# テキストを埋め込みベクトルに変換する関数
def get_embedding(text, model=EMBEDDING_MODEL):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

In [10]:
# テキスト前処理関数
def text_cleanse(text, author_name):
    # テキストを行ごとに分割
    lines = text.split('\n')
    df = pd.DataFrame({'text': lines})

    # 本文の先頭を探す（'-------------------------------------------------------' 区切りの直後から本文が始まる前提）
    head_tx = df[df['text'].str.contains('-------------------------------------------------------')].index.tolist()
    # 本文の末尾を探す（'底本：' の直前に本文が終わる前提）
    atx = df[df['text'].str.contains('底本：')].index.tolist()

    if not head_tx:
        # '-------------------------------------------------------' 区切りが無い場合は、作家名の直後に本文が始まる前提
        head_tx = df[df['text'].str.contains(author_name)].index.tolist()
        if head_tx:
            head_tx_num = head_tx[0] + 1
        else:
            head_tx_num = 0  # 作家名が見つからない場合、テキストの最初から
    else:
        # 2個目の '-------------------------------------------------------' 区切り直後から本文が始まる
        if len(head_tx) > 1:
            head_tx_num = head_tx[1] + 1
        else:
            head_tx_num = head_tx[0] + 1

    if not atx:
        # '底本：' が見つからない場合、テキストの最後まで
        atx_num = len(df)
    else:
        atx_num = atx[0]

    # スライスのコピーを作成
    df_e = df.iloc[head_tx_num:atx_num].copy()

    # 青空文庫の書式削除
    df_e['text'] = df_e['text'].str.replace('《.*?》', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('［.*?］', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('｜', '', regex=True)

    # 字下げ（行頭の全角スペース）を削除
    df_e['text'] = df_e['text'].str.replace('^　+', '', regex=True)

    # 節区切りを削除
    df_e['text'] = df_e['text'].str.replace('^.$', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('^―――.*$', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('^＊＊＊.*$', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('^×××.*$', '', regex=True)

    # 記号、および記号削除によって残ったカッコを削除
    df_e['text'] = df_e['text'].str.replace('―', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('…', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('※', '', regex=True)
    df_e['text'] = df_e['text'].str.replace('「」', '', regex=True)

    # 一文字以下で構成されている行を削除
    df_e = df_e[df_e['text'].str.len() > 1]

    # インデックスをリセット
    df_e = df_e.reset_index(drop=True)

    # テキストを再度結合して一つの文字列に戻す
    cleaned_text = '\n'.join(df_e['text'].tolist())

    return cleaned_text


In [11]:
# テキストファイルを処理してCSVに保存する関数
def process_text_files(input_dir, csv_output_dir, max_tokens=1000):
    for text_file in sorted(input_dir.glob('*.txt')):
        print(f"Processing {text_file.name}...")
        file_path = text_file

        # テキストファイルをShift-JISで読み込む
        with open(file_path, 'r', encoding='shift_jis', errors='ignore') as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines if line.strip()]  # 空行を除去

            # タイトルと著者名を取得
            title = lines[0] if len(lines) > 0 else 'タイトル不明'
            author = lines[1] if len(lines) > 1 else '作者不明'

            # 本文を取得
            text = '\n'.join(lines[2:]) if len(lines) > 2 else ''

            # 前処理を適用
            text = text_cleanse(text, author)

            # テキストをチャンクに分割
            text_chunks = chunk_text(text, max_tokens, encoding)

            # 各チャンクに対して埋め込みを生成し、データフレームに保存
            data = []
            for chunk in text_chunks:
                embedding = get_embedding(chunk)
                data.append({'text': chunk, 'embedding': embedding, 'title': title, 'author': author})

            # DataFrameに変換してCSVファイルに保存（UTF-8で保存）
            df = pd.DataFrame(data, columns=['text', 'embedding', 'title', 'author'])
            csv_file = csv_output_dir / (text_file.stem + '_embeddings.csv')
            df.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"Saved CSV for {text_file.name} to {csv_file}")

            # DataFrameの最初の3行を表示
            print("DataFrameの最初の3行:")
            print(df.head(3))
            # print(tabulate(df.head(3), headers='keys', tablefmt='psql'))

In [12]:
%%time
# テキストファイルを処理してCSVに保存
process_text_files(novel_data_path, csv_output_dir, max_tokens=max_tokens)

Processing 1.txt...
Saved CSV for 1.txt to ../input/embeddings_csv/1_embeddings.csv
DataFrameの最初の3行:
                                                text  \
0  「足音が高いぞ。気付かれてはならん。早くかくれろっ」\n突然、鋭い声があがったかと思うと一緒...   
1  鴨川にのぞんだ裏の座敷へ席をうつして、これから一杯と、最初のその盃を丁度口へ運びかけていたと...   
2  かなたにちりばめて、呼び子の音を求め乍ら、バタバタと駈け近づいた。\n「生憎だな！　薩摩屋敷...   

                                           embedding  title  author  
0  [0.06646344065666199, -0.05843660607933998, 0....  流行暗殺節  佐々木味津三  
1  [0.0706091895699501, -0.0482572540640831, 0.00...  流行暗殺節  佐々木味津三  
2  [0.05724164843559265, -0.0389602854847908, 0.0...  流行暗殺節  佐々木味津三  
Processing 2.txt...
Saved CSV for 2.txt to ../input/embeddings_csv/2_embeddings.csv
DataFrameの最初の3行:
                                                text  \
0  第百版不如帰の巻首に\n不如帰が百版になるので、校正かたがた久しぶりに読んで見た。お坊っちゃ...   
1  �う。\nこなたも引き入れられるるようにうつぶきつ、火鉢にかざせし左手の指環のみ燦然と照り渡...   
2  あわされますぞ、あはははは」と言われしとか。さすがの難波も母の手前、何と挨拶もし兼ねて手持ち...   

                                           embedding   title author 

7.txtがうまく作者名を取得できていないので手作業

In [13]:
pd.read_csv("../input/embeddings_csv/7_embeddings.csv")

,text,embedding,title,author
0,七月二五日。 今週は思いがけない訪問が三つ、わが家にあった。\n最初のものは、井戸掃除職人た...,"[0.04141853749752045, -0.05353863164782524, -0...",死生に関するいくつかの断想,BITS OF LIFE AND DEATH
1,女将が「わたしらば殺したかとですか？」と聞いた。頭領と思しき男が答えた。「殺したかなか！ 金...,"[0.035757020115852356, -0.05810515955090523, 0...",死生に関するいくつかの断想,BITS OF LIFE AND DEATH
2,る。\n（１）「以前の世はどんなものだったの？ どうか見て、教えておくれ」\n一一月一七日。...,"[0.0562724769115448, -0.06713008880615234, 0.0...",死生に関するいくつかの断想,BITS OF LIFE AND DEATH
3,"ところを、女の刀が彼の左肩を切り裂いた。彼は、「人殺しッ！」とは""殺人""を意味すると恐怖の叫...","[0.039558809250593185, -0.049254290759563446, ...",死生に関するいくつかの断想,BITS OF LIFE AND DEATH
4,、私の意地悪な言葉を払いのけるかのように、両手で突然、慌ただしく三度も身振りをした。このかわ...,"[0.02104610577225685, -0.06835035979747772, -0...",死生に関するいくつかの断想,BITS OF LIFE AND DEATH


In [14]:
novel_data_path / "7.txt"

PosixPath('../input/novels/novels/7.txt')

In [15]:
# テキストファイルをShift-JISで読み込む
with open(novel_data_path / "7.txt", 'r', encoding='shift_jis', errors='ignore') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines if line.strip()]  # 空行を除去

    # タイトルと著者名を取得
    title = lines[0] if len(lines) > 0 else 'タイトル不明'
    author = lines[2] if len(lines) > 1 else '作者不明'
    print(title, author)

死生に関するいくつかの断想 小泉八雲　Lafcadio Hearn


In [16]:
df_7 = pd.read_csv("../input/embeddings_csv/7_embeddings.csv")
df_7["title"] = title
df_7["author"] = author

In [17]:
df_7.head()

,text,embedding,title,author
0,七月二五日。 今週は思いがけない訪問が三つ、わが家にあった。\n最初のものは、井戸掃除職人た...,"[0.04141853749752045, -0.05353863164782524, -0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
1,女将が「わたしらば殺したかとですか？」と聞いた。頭領と思しき男が答えた。「殺したかなか！ 金...,"[0.035757020115852356, -0.05810515955090523, 0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
2,る。\n（１）「以前の世はどんなものだったの？ どうか見て、教えておくれ」\n一一月一七日。...,"[0.0562724769115448, -0.06713008880615234, 0.0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
3,"ところを、女の刀が彼の左肩を切り裂いた。彼は、「人殺しッ！」とは""殺人""を意味すると恐怖の叫...","[0.039558809250593185, -0.049254290759563446, ...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
4,、私の意地悪な言葉を払いのけるかのように、両手で突然、慌ただしく三度も身振りをした。このかわ...,"[0.02104610577225685, -0.06835035979747772, -0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn


In [18]:
df_7.to_csv("../input/embeddings_csv/7_embeddings.csv", index=False)
pd.read_csv("../input/embeddings_csv/7_embeddings.csv")

,text,embedding,title,author
0,七月二五日。 今週は思いがけない訪問が三つ、わが家にあった。\n最初のものは、井戸掃除職人た...,"[0.04141853749752045, -0.05353863164782524, -0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
1,女将が「わたしらば殺したかとですか？」と聞いた。頭領と思しき男が答えた。「殺したかなか！ 金...,"[0.035757020115852356, -0.05810515955090523, 0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
2,る。\n（１）「以前の世はどんなものだったの？ どうか見て、教えておくれ」\n一一月一七日。...,"[0.0562724769115448, -0.06713008880615234, 0.0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
3,"ところを、女の刀が彼の左肩を切り裂いた。彼は、「人殺しッ！」とは""殺人""を意味すると恐怖の叫...","[0.039558809250593185, -0.049254290759563446, ...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
4,、私の意地悪な言葉を払いのけるかのように、両手で突然、慌ただしく三度も身振りをした。このかわ...,"[0.02104610577225685, -0.06835035979747772, -0...",死生に関するいくつかの断想,小泉八雲 Lafcadio Hearn
